# Мастерская 2 

Входные данные:
- `base.csv` - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.
- `target.csv` - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков И id товара из base.csv, который максимально похож на него (по мнению экспертов).
- `validation.csv` - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv
- `validation_answer.csv` - правильные ответы к предыдущему файлу.


Задача: 

разработать алгоритм, который для всех товаров из validation.csv предложит несколько вариантов наиболее похожих товаров из base;

оценить качество алгоритма по метрике accuracy@5

*деплой: разработать REST API сервис, который по предложенным данным будем предлагать несколько похожих товаров.


### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import faiss
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

In [2]:
from urllib.parse import urlencode
import os.path
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from catboost import cv, Pool
from sklearn.metrics import roc_auc_score, roc_curve
from joblib import dump, load
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from catboost import cv, Pool
from sklearn.metrics import roc_auc_score, roc_curve
from joblib import dump, load
import matplotlib.pyplot as plt

### Загрузка данных из облака

In [4]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/BBEphK0EHSJ5Jw'
zip_path = '/content/drive/MyDrive/data.zip'
#zip_path = '/content/data.zip'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

NameError: name 'requests' is not defined

In [ ]:
if not os.path.exists(zip_path):
  download_response = requests.get(download_url)
  with open(zip_path, 'wb') as f:
    f.write(download_response.content)

In [ ]:
# Распаковка zip-архива
if not os.path.exists('/content/base.csv'):
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

### Чтение датасетов

Подготовим словарь для корректной загрузки типов данных. По умолчанию загружается float64, мы используем float32.

In [ ]:
dict_base = {}
for i in range(72):
    dict_base[str(i)] = 'float32'

dict_train = dict_base.copy()
dict_train['Target'] = 'str'

Загрузка основного датасета `base.csv`

In [ ]:
df_base = pd.read_csv("/content/base.csv", index_col=0, dtype=dict_base)
df_base.sample(5)

In [ ]:
df_base.shape

In [ ]:
df_base[['0','32','71']].info()

Загрузка датасета с таргетами `train.csv`

In [ ]:
df_train = pd.read_csv("/content/train.csv", index_col=0, dtype=dict_train)
df_train.sample()

In [ ]:
df_train.shape

Загрузка датасета с заданием `validation.csv`

In [ ]:
df_validation = pd.read_csv("/content/validation.csv", index_col=0, dtype=dict_train)
df_validation.sample()

In [ ]:
#df_validation.iloc[0, :].tolist()

In [ ]:
df_validation.shape

Загрузка датасета с ответами `validation_answer.csv`

In [ ]:
df_validation_answer = pd.read_csv("/content/validation_answer.csv", index_col=0, dtype=dict_train)
df_validation_answer.sample()

In [ ]:
df_validation_answer.shape

In [ ]:
df_validation_answer_all = df_validation.join(df_validation_answer)

In [ ]:
df_validation_answer_all.sample()

### Сборка результатов

Напишем функцию для подсчета метрики и сохранения промежуточных результатов.

In [ ]:
overall_scores = {'Кластеры': [],
                  'Поиск в соседних': [],
                  'Опции': [],
                  'accuracy@5': []}

def accuracy_aggregator(targets, idx, base_index, \
                        n_cells, nprobe, comment=''):
  overall_scores['Кластеры'].append(n_cells)
  overall_scores['Поиск в соседних'].append(nprobe)
  overall_scores['Опции'].append(comment)
  acc = 0
  for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])
  result = 100 * acc / len(idx)
  overall_scores['accuracy@5'].append(result)
  print(f'Кластеры: {n_cells}, nprobe: {nprobe}, accuracy@5: {result} %')

## Реализация приближенного поиска

### Baseline поиск

Создаём индекс.

In [ ]:
dims = df_base.shape[1]
n_cells = 100 # количество центроидов
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

Размечаем индекс, добавляем в него вектора.

In [ ]:
idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))

In [ ]:
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}

In [ ]:
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

Осуществляем поиск 5 ближайших соседей и считаем метрику.

In [ ]:
vecs, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), 5)
accuracy_aggregator(targets, idx, base_index, n_cells, 1, 'baseline')

### Поиск с различными параметрами

Посчитаем метрику для различного количества кластеров в индексе и для различного количества кластеров, в которых будет осуществляться поиск. Надо учитывать, что заведомо максимальная метрика может быть получена только за заведомо максимальное время.

In [ ]:
dims = df_base.shape[1] #кол-во признаков
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
k = 5 #кол-во ближайших соседей

n_cells_list = [1000, 500, 200] #кол-во ячеек в индексе
nprobe_list = [1, 2, 10] #кол-во кластеров для поиска

In [ ]:
# неоптимальный перебор параметров по циклу
# нехватает учета времени для каждой итерации
for n_cells in n_cells_list:
  for nprobe in nprobe_list:
    quantizer = faiss.IndexFlatL2(dims)
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))
    idx_l2.nprobe = nprobe
    r, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), k)
    accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'search')

Были и другие эксперименты. Не все из них закончились. Вообще.

### Промежуточные результаты

In [ ]:
my_metrics = pd.DataFrame(overall_scores)
my_metrics

Чем больше кластеров в индексе и быстрее поиск, тем хуже качество поиска. Чем больше соседних кластеров привлекается для поиска, тем качество поиска лучше.

## Оптимизация

### EDA

Посмотрим на распределения признаков в базе `df_base`.

In [ ]:
df_samples = df_base.sample(10000)

In [ ]:
df_samples.hist(figsize=[40, 20], bins=50);

In [ ]:
df_samples = df_train[:5000]

In [ ]:
df_samples.hist(figsize=[40, 20], bins=50);

В `df_train` распределения выглядят аналогичным образом. Отметим, что в столбцах '6', '21', '25', '33', '44', '65', '70' распределения признаков значительно отличаются от нормальных.

### Оптимизация данных

Проведём несколько экспериментов, чтобы понять, как отразятся на метрике удаление отдельных признаков, а также различные варианты масштабирования признаков.

In [ ]:
scaler_MM = MinMaxScaler()
df_base_MM = scaler_MM.fit_transform(df_base)
df_train_MM = scaler_MM.transform(df_train)

In [ ]:
%%time
dims = df_base_MM.shape[1] #кол-во признаков
k = 5 #кол-во ближайших соседей
n_cells = 1000 #кол-во ячеек в индексе
nprobe = 10 #кол-во кластеров для поиска

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
idx_l2.train(np.ascontiguousarray(df_base_MM).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base_MM).astype('float32'))
idx_l2.nprobe = nprobe
r, idx = idx_l2.search(np.ascontiguousarray(df_train_MM).astype('float32'), k)
accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'MinMaxScaler')

In [ ]:
del df_base_MM
del df_train_MM

In [ ]:
unnormal_columns = ['6', '21', '25', '33', '44', '65', '70']
df_base_drop = df_base.copy()
df_train_drop = df_train.copy()

for df in [df_base_drop, df_train_drop]:
  for column in unnormal_columns:
    df.drop(column, axis=1, inplace=True)

In [ ]:
%%time
dims = df_base_drop.shape[1] #кол-во признаков
k = 5 #кол-во ближайших соседей
n_cells = 1000 #кол-во ячеек в индексе
nprobe = 10 #кол-во кластеров для поиска

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
idx_l2.train(np.ascontiguousarray(df_base_drop.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base_drop.values).astype('float32'))
idx_l2.nprobe = nprobe
r, idx = idx_l2.search(np.ascontiguousarray(df_train_drop.values).astype('float32'), k)
accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'Drop Unnormal')

In [ ]:
# important_columns = ['7', '17', '3', '28', '34', '4', '42',
#                      '27', '13', '49', '20', '10', '29', '41',
#                      '55', '62', '58', '24', '16']
important_columns = ['6', '70', '33', '27', '65', '21', '68',
                     '34', '31', '8', '42', '50', '43', '4',
                     '41', '62']
df_base_drop = df_base[important_columns]
df_train_drop = df_train[important_columns]

In [ ]:
%%time
dims = df_base_drop.shape[1] #кол-во признаков
k = 5 #кол-во ближайших соседей
n_cells = 1000 #кол-во ячеек в индексе
nprobe = 10 #кол-во кластеров для поиска

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
idx_l2.train(np.ascontiguousarray(df_base_drop.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base_drop.values).astype('float32'))
idx_l2.nprobe = nprobe
r, idx = idx_l2.search(np.ascontiguousarray(df_train_drop.values).astype('float32'), k)
accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'Drop Unimportant (Catboost-2)')

In [ ]:
scaler_RS = RobustScaler()
df_base_RS = scaler_RS.fit_transform(df_base)
df_train_RS = scaler_RS.transform(df_train)

In [ ]:
%%time
dims = df_base_RS.shape[1] #кол-во признаков
#base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
k = 5 #кол-во ближайших соседей
n_cells = 1000 #кол-во ячеек в индексе
nprobe = 10 #кол-во кластеров для поиска

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
idx_l2.train(np.ascontiguousarray(df_base_RS).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base_RS).astype('float32'))
idx_l2.nprobe = nprobe
r, idx = idx_l2.search(np.ascontiguousarray(df_train_RS).astype('float32'), k)
accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'RobustScaler')

In [ ]:
del df_base_RS
del df_train_RS

In [ ]:
scaler_SS = StandardScaler()
df_base_SS = scaler_SS.fit_transform(df_base)
df_train_SS = scaler_SS.transform(df_train)
scaler_filename = '/content/drive/MyDrive/std_scale_1.bin'
dump(scaler_SS, scaler_filename, compress=True)

df_base_drop_SS = scaler_SS.fit_transform(df_base_drop)
df_train_drop_SS = scaler_SS.transform(df_train_drop)

In [ ]:
del df_base_drop
del df_train_drop

In [ ]:
%%time
dims = df_base_SS.shape[1] #кол-во признаков
k = 5 #кол-во ближайших соседей
n_cells = 1000 #кол-во ячеек в индексе
nprobe = 10 #кол-во кластеров для поиска

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
idx_l2.train(np.ascontiguousarray(df_base_SS).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base_SS).astype('float32'))
idx_l2.nprobe = nprobe
r, idx = idx_l2.search(np.ascontiguousarray(df_train_SS).astype('float32'), k)
accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'StandardScaler')

In [ ]:
%%time
dims = df_base_drop_SS.shape[1] #кол-во признаков
k = 5 #кол-во ближайших соседей
n_cells = 1000 #кол-во ячеек в индексе
nprobe = 10 #кол-во кластеров для поиска

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
idx_l2.train(np.ascontiguousarray(df_base_drop_SS).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base_drop_SS).astype('float32'))
idx_l2.nprobe = nprobe
r, idx = idx_l2.search(np.ascontiguousarray(df_train_drop_SS).astype('float32'), k)
accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, \
                    'StandardScaler, Drop Unimportant')

In [ ]:
del df_base_drop_SS
del df_train_drop_SS
# del df_base_SS
# del df_train_SS

### Промежуточные результаты - 2

In [ ]:
my_metrics = pd.DataFrame(overall_scores)
my_metrics

Наилучшие показатели у данных, отмасштабированных с помощью StandardScaler. Будем работать с этими данными дальше.

Вообще такой большой прирост качества "из коробки" намекает, что работой с признаками можно ещё сильнее улучшить метрику.

## Ранжирование (IMPLEMENTED BUT NOT WORKING)

### Catboost в качестве ранжирующей модели

Соберём большее количество соседей. Для экономии времени была выбрана не самая оптимальная конфигурация.

In [ ]:
# здесь подобрать оптимальные параметры для будущего использования

dims = df_base.shape[1] #кол-во признаков
k = 50 #кол-во ближайших соседей
n_cells = 200 #кол-во ячеек в индексе
nprobe = 20 #кол-во кластеров для поиска
index_filename = f'/content/drive/MyDrive/idx_l2_{n_cells}_{nprobe}.index'

Сохраним индекс на диске. В будущем можно будет им пользоваться, не проводя предварительные вычисления.

In [ ]:
%%time
if not os.path.exists(index_filename):
  quantizer = faiss.IndexFlatL2(dims)
  idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
  idx_l2.train(np.ascontiguousarray(df_base_SS).astype('float32'))
  idx_l2.add(np.ascontiguousarray(df_base_SS).astype('float32'))
  idx_l2.nprobe = nprobe
  r, idx = idx_l2.search(np.ascontiguousarray(df_train_SS).astype('float32'), k)
  accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'StandardScaler, 50 Neighbours')
  faiss.write_index(idx_l2, index_filename)
else:
  idx_l2 = faiss.read_index(index_filename)

In [ ]:
%%time
r, idx = idx_l2.search(np.ascontiguousarray(df_train_SS).astype('float32'), 50)
accuracy_aggregator(targets, idx, base_index, n_cells, nprobe, 'StandardScaler, 50 Neighbours')

In [ ]:
my_metrics = pd.DataFrame(overall_scores)
my_metrics.tail(1)

In [ ]:
len(idx)

In [ ]:
len(idx[0])

In [ ]:
targets.head()

In [ ]:
# формирование списка названия колонок для обучающего датафрейма
columns=df_base.columns.tolist()
all_columns = columns
new_columns = [f'{col}_search' for col in columns]
all_columns.extend(new_columns)
all_columns.extend(['is_neighbour'])

In [ ]:
# Далее идет совершенно неправильный способ задания обучающего датафрейма.
# df_base_cat = pd.DataFrame(columns=all_columns)
# columns=df_base.columns.tolist()
# df_base_cat

In [ ]:
# инициализация матриц для внесения сравниваемых признаков
np_base_SS = np.empty(72)
np_train_SS = np.empty(72)

results = []
base_names = []

In [ ]:
%%time
sycle = 0
# Берём ограниченный объём данных.
for target, el in zip(targets.values.tolist()[:2000], idx.tolist()[:2000]):
  for r in el:
    #если делать так, всё очень медленно
    #df_base_cat.loc[sycle, columns] = df_base_SS[r]
    #df_base_cat.loc[sycle, new_columns] = df_train_SS[sycle]
    #df_base_cat.loc[sycle, 'result'] = base_index[r]

    if target == base_index[r]:
      #df_base_cat.loc[sycle, 'is_neighbour'] = 1
      results.append(1)
    else:
      #df_base_cat.loc[sycle, 'is_neighbour'] = 0
      results.append(0)

    np_base_SS = np.vstack([np_base_SS, df_base_SS[r]])
    np_train_SS = np.vstack([np_train_SS, df_train_SS[sycle]])
    base_names.append(base_index[r])
    sycle+=1

sycle

In [ ]:
sum(results)

In [ ]:
# соединяем матрицы, удаляем первую строку (возникла при инициализации)
cat_array = np.hstack([np_base_SS, np_train_SS])
cat_array = np.delete(cat_array, (0), axis=0)
cat_array.shape

In [ ]:
# добаляем столбец с таргетом
cat_array = np.insert (cat_array, 144, results, axis=1)

In [ ]:
# матрица становится датафреймом
df_base_cat = pd.DataFrame(data=cat_array, columns=all_columns, index=base_names)
df_base_cat.shape

In [ ]:
# df_base_cat['result'] = base_names
# df_base_cat = df_base_cat.set_index('result')

In [ ]:
# df_base_cat.shape

In [ ]:
df_base_cat[df_base_cat['is_neighbour']==1].sample()

In [ ]:
# Очень мало целевого признака
df_base_cat['is_neighbour'].sum()/df_base_cat.shape[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_base_cat.drop(['is_neighbour'] , axis=1),
    df_base_cat['is_neighbour'],
    test_size=0.25,
    random_state=2007,
    stratify=df_base_cat['is_neighbour']
    )

In [ ]:
X_train.sample()

In [ ]:
cat_features = []

In [ ]:
train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features
                 )

In [ ]:
test_data = Pool(data=X_test,
                  label=y_test,
                  cat_features=cat_features
                 )

In [ ]:
params = {'cat_features': cat_features,
          'eval_metric': 'AUC',
          'loss_function': 'Logloss',
          #'learning_rate': 0.01,
          'random_seed': 2007,
          'verbose':100
          }

In [ ]:
cv_data = cv(
    params = params,
    pool = train_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=True,
    verbose=False,
    #early_stopping_rounds=200
)

In [ ]:
model = CatBoostClassifier(**params)

In [ ]:
model.fit(train_data)

In [ ]:
# может быть это будет полезно для оптимизации поиска.
# проверено. В первом случае помогло, во втором - наоборот
model.get_feature_importance(prettified=True).head(20)

In [ ]:
# Сохраним модель для будущего использования
model_filename = f'/content/drive/MyDrive/cbm_big.cbm'
model.save_model(model_filename,
           format="cbm",
           export_parameters=None,
           pool=None)

In [ ]:
y_pr = model.predict_proba(test_data)[:, 1]
#np.argsort(y_pr)

In [ ]:
X_test['is_neighbour'] = y_test
X_test['is_neighbour_proba'] = y_pr
roc_auc_score(X_test['is_neighbour'],X_test['is_neighbour_proba'])

In [ ]:
fpr, tpr, thresholds = roc_curve(X_test['is_neighbour'], X_test['is_neighbour_proba'])
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.ylim([0.0, 1.0])
plt.xlim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

Модель требует дополнительной настройки и в данном виде неприменима для улучшения метрики.

### Промежуточные результаты - 3

In [ ]:
my_metrics = pd.DataFrame(overall_scores)
my_metrics

Как показывает тестирование, модель в состоянии указать на наиболее важные признаки, которым можно будет уделять дополнительное внимание.

## Решение задачи

Загрузим сохранённые ранее индекс, скейлер и модель и проведём поиск 5 ближайших соседей для всех товаров из `validation.csv`.

In [ ]:
idx_l2 = faiss.read_index('/content/drive/MyDrive/idx_l2_200_20.index')
nprobe = idx_l2.nprobe

In [ ]:
# ответы у нас уже есть
answers = df_validation_answer.squeeze()
# не забываем масштабирование
scaler_SS=load('/content/drive/MyDrive/std_scale_1.bin')
df_validation_SS = scaler_SS.transform(df_validation)

In [ ]:
# загрузка модели CatBoostClassifier
model = CatBoostClassifier()
model.load_model('/content/drive/MyDrive/cbm_big.cbm')
print(model.get_params())

Найдём 20 ближайших соседей для валидационной выборки. Посчитаем сначала метрику без ранжирующей модели.

In [ ]:
%%time
r, idx = idx_l2.search(np.ascontiguousarray(df_validation_SS).astype('float32'), 20)
idx_cat = idx # для ранжирующей модели возьмем всех найденных 20 соседей
idx = np.delete(idx, list(range(5, 20)), 1)
idx.shape

In [ ]:
accuracy_aggregator(
    answers, idx, base_index, 200, nprobe, 'Validation, 5 Neigbours')

In [ ]:
accuracy_aggregator(
    answers, idx_cat, base_index, 200, nprobe, 'Validation, 20 Neigbours')

In [ ]:
idx_cat.shape

Теперь с помощью ранжирующей модели выберем из 20 соседей лучших 5 по мнению модели. Посчитаем метрику.

In [ ]:
# подготовка к инициализации датафрейма df_base_cat,
# который подаётся на вход модели Catboost
columns=df_base.columns.tolist()
all_columns = columns
new_columns = [f'{col}_search' for col in columns]
all_columns.extend(new_columns)
df_base_cat = pd.DataFrame(columns=all_columns)
columns=df_base.columns.tolist()
idx_cat_cut = np.empty(5, dtype=int)

In [ ]:
# здесь опять медленный способ работы, нужно исправить.
# опять работа на ограниченном наборе данных
for step in range(20000):
  df_base_cat = pd.DataFrame(columns=all_columns)
  cycle = 0
  for el in idx_cat[step]:
    df_base_cat.loc[cycle, columns] = df_base_SS[el]
    df_base_cat.loc[cycle, new_columns] = df_validation_SS[step]
    cycle += 1
  z_pr = model.predict_proba(df_base_cat)[:, 1]
  idx_cat_cut = np.vstack([idx_cat_cut, np.delete(idx_cat[step], np.argsort(z_pr)[:15], 0)])

In [ ]:
# удаление лишней первой строки
idx_cat_cut = np.delete(idx_cat_cut, 0, axis=0)

In [ ]:
idx_cat_cut.shape

In [ ]:
accuracy_aggregator(
    answers[:20000], idx_cat_cut, base_index, 200, nprobe, 'Validation, 5 Neigbours, CatBoost')

In [ ]:
my_metrics = pd.DataFrame(overall_scores)
my_metrics.tail(3)

## Выводы

В проекте решалась **задача разработки алгоритма, который для всех товаров из `validation.csv` предложит несколько вариантов наиболее похожих товаров из `base.csv`**.

При этом:
- `base.csv` - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.
- `validation.csv` - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv

Для подбора оптимальных параметров приближенного поиска использовался обучающий датасет `target.csv`. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …), вектор признаков и id товара из base.csv, который максимально похож на него (по мнению экспертов).

- При решении задачи была использована библиотека для приближённого поиска ближайших соседей `FAISS`.

- Проведён исследовательский анализ данных. В целях повышения метрики произведено масштабирование признаков с использованием `StandardScaler`.

- Создан, обучен и сохранён на диск индекс для `FAISS` с оптимальными в рамках данного исследования параметрами.

- В качестве демонстрации подхода обучена ранжирующая модель CatBoost. В дальнейшем модель позволит дополнительно оптимизировать поиск, а также выделить наиболее значимые признаки.

- Время поиска 5 ближайших соседей для датасета `validation.csv` на этом индексе составило около **15 минут** (Colab, без подписки).

- Качество предложенного алгоритма по метрике `accuracy@5` для датасета `validation.csv` составило **64,8%** (без применения ранжирующей модели).

- Сохранённые индекс и модель позволяют реализовать развёртывание решения в виде микросервиса.


### Что нужно улучшить

- Для выбора оптимального индекса надо оценить время работы для каждого из них
- Дальнейшая работа с признаками должна повысить метрику
- От ранжирующей модели нужно получить большее качество, после этого её нужно применять для отбора более качественных соседей (если это будет оправдано с точки зрения производительности)
- Полноценно реализовать решение в качестве микросервиса